In [6]:

# CFB Dynasty Roster Analysis and Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import glob
import os
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Libraries imported successfully!")
print("📊 Ready for CFB Dynasty roster analysis and visualization")

ModuleNotFoundError: No module named 'seaborn'

# 🏈 CFB Dynasty Roster Analysis Dashboard

This notebook provides comprehensive analysis and visualization of your CFB Dynasty roster data, including:

- **Player Valuations** - Calculate player value based on ratings, development traits, and remaining years
- **Roster Composition** - Analyze position strength and depth across your roster
- **Recruiting Strategy** - Identify positions of need and priority for recruiting
- **Interactive Visualizations** - Charts and graphs to better understand your roster

---

In [ ]:
# Configuration and Constants

# Define minimum and ideal roster sizes per position
# TODO: Update positions and archetypes for CFB 26
default_position_requirements = {
    'QB': {'min': 3, 'ideal': 4, 'archetypes': {'FIELD GENERAL': 1, 'IMPROVISER': 0.75, 'SCRAMBLER': 0.25}},
    'HB': {'min': 4, 'ideal': 6, 'archetypes': {'ELUSIVE BACK': 0.75, 'POWER BACK': 0.75, 'RECEIVING BACK': 0.5}},
    'FB': {'min': 0, 'ideal': 0, 'archetypes': {'UTILITY': 0, 'BLOCKING': 0}},
    'WR': {'min': 6, 'ideal': 8, 'archetypes': {'DEEP THREAT': 0.75, 'PHYSICAL': 0.75, 'ROUTE RUNNER': 0.75}},
    'TE': {'min': 3, 'ideal': 4, 'archetypes': {'VERTICAL THREAT': 1, 'POSSESSION': 0.25, 'BLOCKING': 0.25}},
    'LT': {'min': 3, 'ideal': 4, 'archetypes': {'PASS PROTECTOR': 1, 'AGILE': 0.8, 'POWER': 0.6}},
    'LG': {'min': 3, 'ideal': 4, 'archetypes': {'PASS PROTECTOR': 0.6, 'AGILE': 1, 'POWER': 1}},
    'C':  {'min': 3, 'ideal': 4, 'archetypes': {'PASS PROTECTOR': 0.5, 'AGILE': 1, 'POWER': 1}},
    'RG': {'min': 3, 'ideal': 4, 'archetypes': {'PASS PROTECTOR': 0.6, 'AGILE': 1, 'POWER': 1}},
    'RT': {'min': 3, 'ideal': 4, 'archetypes': {'PASS PROTECTOR': 0.8, 'AGILE': 0.8, 'POWER': 0.75}},
    'LE': {'min': 3, 'ideal': 4, 'archetypes': {'POWER RUSHER': 0.9, 'SPEED RUSHER': 0.9, 'RUN STOPPER': 1}},
    'RE': {'min': 3, 'ideal': 4, 'archetypes': {'POWER RUSHER': 0.9, 'SPEED RUSHER': 1, 'RUN STOPPER': 1}},
    'DT': {'min': 3, 'ideal': 4, 'archetypes': {'POWER RUSHER': 0.9, 'SPEED RUSHER': 0.8, 'RUN STOPPER': 1}},
    'LOLB': {'min': 3, 'ideal': 4, 'archetypes': {'POWER RUSHER': 1, 'RUN STOPPER': 1, 'PASS COVERAGE': 0.1}},
    'MLB': {'min': 3, 'ideal': 4, 'archetypes': {'FIELD GENERAL': 1, 'RUN STOPPER': 1, 'PASS COVERAGE': 1}},
    'ROLB': {'min': 3, 'ideal': 4, 'archetypes': {'POWER RUSHER': 1, 'RUN STOPPER': 1, 'PASS COVERAGE': 0.1}},
    'CB': {'min': 5, 'ideal': 7, 'archetypes': {'MAN TO MAN': 1, 'ZONE': 0.75, 'SLOT': 0.5}},
    'FS': {'min': 2, 'ideal': 3, 'archetypes': {'ZONE': 1, 'HYBRID': 0.75, 'RUN SUPPORT': 0.5}},
    'SS': {'min': 2, 'ideal': 3, 'archetypes': {'ZONE': 0.25, 'HYBRID': 0.75, 'RUN SUPPORT': 0.75}},
    'K': {'min': 1, 'ideal': 1, 'archetypes': {'ACCURATE': 0.75, 'POWER': 0.75}},
    'P': {'min': 1, 'ideal': 1, 'archetypes': {'ACCURATE': 0.75, 'POWER': 0.75}}
}

# Development trait multipliers
dev_trait_multipliers = {
    'NORMAL': 1.00,
    'IMPACT': 1.10,
    'STAR': 1.25,
    'ELITE': 1.50
}

# Remaining years of development for different player years
remaining_years = {
    'FR': 3, 'SO': 2, 'JR': 1, 'SR': 0,
    'FR (RS)': 3, 'SO (RS)': 2, 'JR (RS)': 1, 'SR (RS)': 0
}

# Redshirt discount and starter counts
rs_discount = 0.05

starters_count = {
    'QB': 1, 'HB': 2, 'FB': 1, 'WR': 3, 'TE': 1,
    'LT': 1, 'LG': 1, 'C': 1, 'RG': 1, 'RT': 1,
    'LE': 1, 'RE': 1, 'DT': 2, 'LOLB': 1, 'MLB': 1, 'ROLB': 1,
    'CB': 2, 'FS': 1, 'SS': 1, 'K': 1, 'P': 1
}

print("📋 Configuration loaded successfully!")
print(f"📊 Tracking {len(default_position_requirements)} positions")
print(f"🎯 {len(dev_trait_multipliers)} development trait levels defined")

📋 Configuration loaded successfully!
📊 Tracking 21 positions
🎯 4 development trait levels defined


In [ ]:
# Core Analysis Functions

def calculate_player_value(row, dev_trait_multipliers=dev_trait_multipliers, rs_discount=rs_discount):
    """Calculate player value based on base overall, development trait, and remaining years."""
    redshirt_discount = rs_discount if "(RS)" in row['YEAR'] else 0
    dev_multiplier = dev_trait_multipliers.get(row['DEV TRAIT'], 1.00)
    remaining_dev_years = remaining_years.get(row['YEAR'], 0)
    value = round(row['BASE OVERALL'] * dev_multiplier * (1 + remaining_dev_years / 4) * (1 - redshirt_discount), 2)
    return value

def player_status(row):
    """Determine if player is safe, at risk, or on the cut list."""
    value = row['VALUE']
    year = row['YEAR']
    best_at_position = row['Best at Position']

    if year in ['SR', 'SR (RS)']:
        return 'GRADUATING'
    elif best_at_position:
        return 'SAFE'

    elif value < 100:
        return 'CUT'
    elif value >= 100 and value <= 125:
        return 'AT RISK'
    else:
        return 'SAFE'

def calculate_position_grade(avg_value):
    """Calculate letter grade based on average position value."""
    if avg_value >= 150: return 'A+'
    elif avg_value >= 140: return 'A'
    elif avg_value >= 130: return 'A-'
    elif avg_value >= 120: return 'B+'
    elif avg_value >= 110: return 'B'
    elif avg_value >= 100: return 'B-'
    elif avg_value >= 90: return 'C+'
    elif avg_value >= 80: return 'C'
    elif avg_value >= 70: return 'C-'
    else: return 'F'

def calculate_blended_measure(df, position):
    """Calculate blended measure of starters and backups (70% starters, 30% backups)."""
    starters_num = starters_count.get(position, 1)
    position_df = df[df['POSITION'] == position].sort_values(by='VALUE', ascending=False)

    starters = position_df.head(starters_num)
    backups = position_df.tail(len(position_df) - starters_num)

    starters_avg = starters['VALUE'].mean() if len(starters) > 0 else 0
    backups_avg = backups['VALUE'].mean() if len(backups) > 0 else 0

    blended_value = round(0.7 * starters_avg + 0.3 * backups_avg, 2)
    return blended_value

def determine_priority(row):
    """Determine recruiting priority for each position."""
    if row['Current Count'] < row['Min Required']:
        return 'HIGH'
    elif row['Grade'] in ['D', 'F']:
        return 'HIGH'
    elif row['Grade'] in ['C']:
        return 'MEDIUM'
    else:
        return 'LOW'

print("✅ Analysis functions defined successfully!")
print("🔧 Ready to process roster data")

✅ Analysis functions defined successfully!
🔧 Ready to process roster data


## 📂 Data Loading

Load your roster CSV file from the Downloads folder. The script will automatically detect roster files.

In [ ]:
# Load Roster Data
downloads_folder = os.path.expanduser('~/Downloads')
data_folder = os.path.join(downloads_folder, 'cfb_dynasty_data')

# Find roster files
roster_files = glob.glob(os.path.join(downloads_folder, '*[Rr]oster.csv'))

if not roster_files:
    print("❌ No roster CSV files found in Downloads folder")
    print("📁 Make sure your roster file is in ~/Downloads/ and contains 'roster' in the filename")
else:
    print(f"📁 Found {len(roster_files)} roster file(s):")
    for i, file in enumerate(roster_files):
        print(f"  {i+1}. {os.path.basename(file)}")

    # Load the first roster file found
    roster_path = roster_files[0]
    print(f"\n📊 Loading: {os.path.basename(roster_path)}")

    try:
        roster_df = pd.read_csv(roster_path)
        print(f"✅ Successfully loaded {len(roster_df)} players")
        print(f"📋 Columns: {list(roster_df.columns)}")

        # Display basic info about the roster
        print(f"\n📈 Quick Stats:")
        print(f"  • Total Players: {len(roster_df)}")
        print(f"  • Positions: {roster_df['POSITION'].nunique()}")
        print(f"  • Years: {', '.join(sorted(roster_df['YEAR'].unique()))}")

    except Exception as e:
        print(f"❌ Error loading roster file: {e}")
        roster_df = None

📁 Found 1 roster file(s):
  1. Rice Dynasty - 2026 raw roster.csv

📊 Loading: Rice Dynasty - 2026 raw roster.csv
✅ Successfully loaded 85 players
📋 Columns: ['RS', 'FIRST NAME', 'LAST NAME', 'YEAR', 'POSITION', 'OVERALL', 'BASE OVERALL', 'ARCHETYPE', 'DEV TRAIT']

📈 Quick Stats:
  • Total Players: 85
  • Positions: 20
  • Years: FR, FR (RS), JR, JR (RS), SO, SO (RS), SR, SR (RS)


In [ ]:
# Process Roster Data
if roster_df is not None:
    # Ensure required columns exist
    required_columns = [
        'RS', 'FIRST NAME', 'LAST NAME', 'YEAR', 'POSITION', 'OVERALL', 'BASE OVERALL',
        'ARCHETYPE', 'DEV TRAIT', 'VALUE', 'STATUS', 'Best at Position', 'Current Count', 'Min Required', 'Grade'
    ]

    missing_columns = [col for col in required_columns if col not in roster_df.columns]
    if missing_columns:
        print(f"⚠️  Missing required columns: {missing_columns}")
        print("📝 Adding missing columns with default values...")

        # Add missing columns with defaults
        for col in missing_columns:
            if col in ['CUT', 'REDSHIRT']:
                roster_df[col] = False
            elif col == 'DRAFTED':
                roster_df[col] = None
            else:
                roster_df[col] = ''

    # Calculate player values
    roster_df['VALUE'] = roster_df.apply(calculate_player_value, axis=1)

    # Fill missing archetypes
    roster_df['ARCHETYPE'] = roster_df['ARCHETYPE'].fillna('')

    # Determine best player at each position
    roster_df['Best at Position'] = roster_df.groupby('POSITION')['OVERALL'].transform(lambda x: x == x.max())

    # Apply player status
    roster_df['STATUS'] = roster_df.apply(player_status, axis=1)

    # Sort roster
    position_order = ['QB', 'HB', 'WR', 'TE', 'LT', 'LG', 'C', 'RG', 'RT',
                     'LE', 'RE', 'DT', 'LOLB', 'MLB', 'ROLB', 'CB', 'FS', 'SS', 'K', 'P', 'ATH']
    roster_df['POSITION'] = pd.Categorical(roster_df['POSITION'], categories=position_order, ordered=True)
    roster_df = roster_df.sort_values(by=['POSITION', 'OVERALL'], ascending=[True, False])

    print("✅ Roster data processed successfully!")
    print(f"💰 Player values calculated")
    print(f"📊 Player status determined")

    # Show sample of processed data
    print(f"\n📋 Sample of processed roster:")
    display_cols = [
        'RS', 'FIRST NAME', 'LAST NAME', 'YEAR', 'POSITION', 'OVERALL',
        'BASE OVERALL', 'ARCHETYPE', 'DEV TRAIT', 'STATUS'
    ]
    sample_df = roster_df[display_cols].head(10)
    print(sample_df.to_string(index=False))

⚠️  Missing required columns: ['VALUE', 'STATUS', 'Best at Position', 'Current Count', 'Min Required', 'Grade']
📝 Adding missing columns with default values...
✅ Roster data processed successfully!
💰 Player values calculated
📊 Player status determined

📋 Sample of processed roster:
 RS FIRST NAME    LAST NAME    YEAR POSITION  OVERALL  BASE OVERALL            ARCHETYPE DEV TRAIT  STATUS
NaN      CHASE      JENKINS SO (RS)       QB       71            71          DUAL THREAT    NORMAL    SAFE
 RS    PATRICK  CRAYTON JR.      FR       QB       71            71          DUAL THREAT    NORMAL    SAFE
NaN         AJ      PADGETT JR (RS)       QB       70            70        POCKET PASSER    NORMAL     CUT
NaN       DREW    DEVILLIER FR (RS)       QB       70            70        POCKET PASSER    IMPACT    SAFE
NaN        MAX    BALTHAZAR JR (RS)       HB       79            79  EAST/WEST PLAYMAKER    NORMAL    SAFE
NaN    QUINTON      JACKSON JR (RS)       HB       78            78  EAST/W

In [ ]:
# Generate Recruiting Plan
if roster_df is not None:
    # Calculate next season counts (excluding graduating players)
    next_season_counts = roster_df[roster_df['STATUS'] != 'GRADUATING'].groupby('POSITION').size()

    # Calculate blended values for each position
    blended_values = {pos: calculate_blended_measure(roster_df, pos) for pos in default_position_requirements.keys()}

    # Create recruiting plan DataFrame
    recruiting_plan = pd.DataFrame({
        'Position': default_position_requirements.keys(),
        'Current Count': [next_season_counts.get(pos, 0) for pos in default_position_requirements.keys()],
        'Min Required': [default_position_requirements[pos]['min'] for pos in default_position_requirements.keys()],
        'Ideal Count': [default_position_requirements[pos]['ideal'] for pos in default_position_requirements.keys()],
        'Blended Value': [blended_values[pos] for pos in default_position_requirements.keys()],
        'Grade': [calculate_position_grade(blended_values[pos]) for pos in default_position_requirements.keys()]
    }).fillna(0)

    # Determine priority
    recruiting_plan['Priority'] = recruiting_plan.apply(determine_priority, axis=1)

    # Calculate need (difference between current and ideal)
    recruiting_plan['Need'] = recruiting_plan['Ideal Count'] - recruiting_plan['Current Count']
    recruiting_plan['Need'] = recruiting_plan['Need'].clip(lower=0)  # Don't show negative needs

    print("✅ Recruiting plan generated!")
    print(f"\n📋 Recruiting Priorities Summary:")

    # Show high priority positions
    high_priority = recruiting_plan[recruiting_plan['Priority'] == 'HIGH'].sort_values('Need', ascending=False)
    if len(high_priority) > 0:
        print(f"\n🚨 HIGH PRIORITY positions ({len(high_priority)}):")
        for _, row in high_priority.iterrows():
            print(f"  • {row['Position']}: {int(row['Current Count'])}/{int(row['Min Required'])} (Grade: {row['Grade']})")

    medium_priority = recruiting_plan[recruiting_plan['Priority'] == 'MEDIUM']
    if len(medium_priority) > 0:
        print(f"\n⚠️  MEDIUM PRIORITY positions ({len(medium_priority)}):")
        for _, row in medium_priority.iterrows():
            print(f"  • {row['Position']}: Grade {row['Grade']} (Value: {row['Blended Value']:.1f})")

    print(f"\n📊 Full recruiting plan available in 'recruiting_plan' DataFrame")

✅ Recruiting plan generated!

📋 Recruiting Priorities Summary:

🚨 HIGH PRIORITY positions (9):
  • C: 0/3 (Grade: F)
  • LE: 0/3 (Grade: F)
  • RE: 0/3 (Grade: F)
  • LOLB: 0/3 (Grade: F)
  • MLB: 0/3 (Grade: F)
  • ROLB: 0/3 (Grade: F)
  • SS: 1/2 (Grade: C)
  • P: 0/1 (Grade: F)
  • FB: 0/0 (Grade: F)

📊 Full recruiting plan available in 'recruiting_plan' DataFrame


## 📊 Data Visualizations

Interactive charts and graphs to analyze your roster composition, player values, and recruiting needs.

In [ ]:
# 1. Roster Composition by Position
if roster_df is not None:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Current Roster by Position', 'Position Strength Grades',
                       'Player Status Distribution', 'Development Traits'),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "pie"}, {"type": "pie"}]]
    )

    # Subplot 1: Current roster by position
    position_counts = roster_df['POSITION'].value_counts().reindex(position_order).fillna(0)
    fig.add_trace(
        go.Bar(x=position_counts.index, y=position_counts.values,
               name="Current Count", marker_color='lightblue'),
        row=1, col=1
    )

    # Subplot 2: Position grades
    grade_colors = {'A+': '#2E8B57', 'A': '#32CD32', 'A-': '#90EE90',
                   'B+': '#FFD700', 'B': '#FFA500', 'B-': '#FF8C00',
                   'C+': '#FF6347', 'C': '#FF4500', 'C-': '#DC143C', 'F': '#8B0000'}

    recruiting_plan_sorted = recruiting_plan.sort_values('Blended Value', ascending=True)
    colors = [grade_colors.get(grade, '#808080') for grade in recruiting_plan_sorted['Grade']]

    fig.add_trace(
        go.Bar(x=recruiting_plan_sorted['Blended Value'],
               y=recruiting_plan_sorted['Position'],
               orientation='h',
               name="Position Strength",
               marker_color=colors,
               text=recruiting_plan_sorted['Grade'],
               textposition='auto'),
        row=1, col=2
    )

    # Subplot 3: Player status distribution
    status_counts = roster_df['STATUS'].value_counts()
    status_colors = {'SAFE': '#2E8B57', 'AT RISK': '#FFD700', 'CUT': '#DC143C', 'GRADUATING': '#808080'}
    fig.add_trace(
        go.Pie(labels=status_counts.index, values=status_counts.values,
               name="Status", marker_colors=[status_colors.get(status, '#808080') for status in status_counts.index]),
        row=2, col=1
    )

    # Subplot 4: Development traits
    dev_counts = roster_df['DEV TRAIT'].value_counts()
    dev_colors = {'ELITE': '#2E8B57', 'STAR': '#32CD32', 'IMPACT': '#FFD700', 'NORMAL': '#808080'}
    fig.add_trace(
        go.Pie(labels=dev_counts.index, values=dev_counts.values,
               name="Dev Traits", marker_colors=[dev_colors.get(trait, '#808080') for trait in dev_counts.index]),
        row=2, col=2
    )

    fig.update_layout(height=800, title_text="🏈 Roster Overview Dashboard", showlegend=False)
    fig.update_xaxes(title_text="Position", row=1, col=1)
    fig.update_yaxes(title_text="Player Count", row=1, col=1)
    fig.update_xaxes(title_text="Blended Value", row=1, col=2)

    fig.show()

    print("📊 Roster composition visualization created!")

📊 Roster composition visualization created!


In [ ]:
# 2. Player Value Distribution
if roster_df is not None:
    fig = px.box(roster_df, x='POSITION', y='VALUE',
                 color='STATUS',
                 title='📈 Player Value Distribution by Position',
                 color_discrete_map={'SAFE': '#2E8B57', 'AT RISK': '#FFD700',
                                   'CUT': '#DC143C', 'GRADUATING': '#808080'})

    fig.update_layout(
        xaxis_title="Position",
        yaxis_title="Player Value",
        height=600,
        xaxis={'categoryorder':'array', 'categoryarray':position_order}
    )

    fig.show()

    # Additional value statistics
    print(f"💰 Player Value Statistics:")
    print(f"  • Average Value: {roster_df['VALUE'].mean():.1f}")
    print(f"  • Highest Value: {roster_df['VALUE'].max():.1f} ({roster_df.loc[roster_df['VALUE'].idxmax(), 'FIRST NAME']} {roster_df.loc[roster_df['VALUE'].idxmax(), 'LAST NAME']})")
    print(f"  • Lowest Value: {roster_df['VALUE'].min():.1f}")
    print(f"  • Players > 150 Value: {len(roster_df[roster_df['VALUE'] > 150])}")
    print(f"  • Players at Risk (100-125): {len(roster_df[roster_df['STATUS'] == 'AT RISK'])}")
    print(f"  • Players to Cut (<100): {len(roster_df[roster_df['STATUS'] == 'CUT'])}")

💰 Player Value Statistics:
  • Average Value: 91.1
  • Highest Value: 129.5 (TYVONN BYARS)
  • Lowest Value: 63.6
  • Players > 150 Value: 0
  • Players at Risk (100-125): 17
  • Players to Cut (<100): 32


In [ ]:
# 3. Recruiting Priority Heatmap
if roster_df is not None and recruiting_plan is not None:
    # Create a matrix for the heatmap
    priority_map = {'HIGH': 3, 'MEDIUM': 2, 'LOW': 1}
    recruiting_plan['Priority_Numeric'] = recruiting_plan['Priority'].map(priority_map)

    # Create heatmap data
    heatmap_data = recruiting_plan.pivot_table(
        index='Position',
        values=['Priority_Numeric', 'Need', 'Blended Value'],
        aggfunc='first'
    ).fillna(0)

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Recruiting Priority', 'Position Need', 'Current Strength'),
        horizontal_spacing=0.1
    )

    # Priority heatmap
    fig.add_trace(
        go.Heatmap(
            z=[heatmap_data['Priority_Numeric'].values],
            x=heatmap_data.index,
            y=['Priority'],
            colorscale=[[0, 'green'], [0.5, 'yellow'], [1, 'red']],
            showscale=False,
            text=[[recruiting_plan.set_index('Position').loc[pos, 'Priority'] for pos in heatmap_data.index]],
            texttemplate="%{text}",
            textfont={"size": 10}
        ),
        row=1, col=1
    )

    # Need heatmap
    fig.add_trace(
        go.Heatmap(
            z=[heatmap_data['Need'].values],
            x=heatmap_data.index,
            y=['Need'],
            colorscale='Reds',
            showscale=False,
            text=[[f"{int(val)}" for val in heatmap_data['Need'].values]],
            texttemplate="%{text}",
            textfont={"size": 10}
        ),
        row=1, col=2
    )

    # Strength heatmap
    fig.add_trace(
        go.Heatmap(
            z=[heatmap_data['Blended Value'].values],
            x=heatmap_data.index,
            y=['Strength'],
            colorscale='RdYlGn',
            showscale=True,
            text=[[f"{val:.0f}" for val in heatmap_data['Blended Value'].values]],
            texttemplate="%{text}",
            textfont={"size": 10}
        ),
        row=1, col=3
    )

    fig.update_layout(
        title='🎯 Recruiting Strategy Heatmap',
        height=400,
        showlegend=False
    )

    # Rotate x-axis labels
    for i in range(1, 4):
        fig.update_xaxes(tickangle=45, row=1, col=i)

    fig.show()

    print("🎯 Recruiting priority heatmap created!")
    print("\n📋 Quick Reference:")
    print("  🔴 RED = High Priority/High Need")
    print("  🟡 YELLOW = Medium Priority")
    print("  🟢 GREEN = Low Priority/Strong Position")

🎯 Recruiting priority heatmap created!

📋 Quick Reference:
  🔴 RED = High Priority/High Need
  🟡 YELLOW = Medium Priority
  🟢 GREEN = Low Priority/Strong Position


In [ ]:
# 4. Year Class Distribution and Future Outlook
if roster_df is not None:
    # Create year class analysis
    year_order = ['FR', 'SO', 'JR', 'SR', 'FR (RS)', 'SO (RS)', 'JR (RS)', 'SR (RS)']

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Current Year Distribution', 'Position by Year Class',
                       'Development Potential by Year', 'Future Roster Projection'),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )

    # Year distribution
    year_counts = roster_df['YEAR'].value_counts().reindex(year_order).fillna(0)
    fig.add_trace(
        go.Bar(x=year_counts.index, y=year_counts.values,
               name="Players by Year", marker_color='lightcoral'),
        row=1, col=1
    )

    # Position by year (scatter plot)
    for pos in position_order[:10]:  # Show top 10 positions to avoid clutter
        pos_data = roster_df[roster_df['POSITION'] == pos]
        if len(pos_data) > 0:
            fig.add_trace(
                go.Scatter(x=pos_data['YEAR'], y=[pos] * len(pos_data),
                          mode='markers', name=pos,
                          marker=dict(size=8, opacity=0.7)),
                row=1, col=2
            )

    # Development potential by year
    year_dev_potential = roster_df.groupby('YEAR')['VALUE'].mean().reindex(year_order).fillna(0)
    fig.add_trace(
        go.Bar(x=year_dev_potential.index, y=year_dev_potential.values,
               name="Avg Value by Year", marker_color='lightgreen'),
        row=2, col=1
    )

    # Future projection (non-graduating players)
    future_roster = roster_df[roster_df['STATUS'] != 'GRADUATING']
    future_counts = future_roster['POSITION'].value_counts().reindex(position_order).fillna(0)
    fig.add_trace(
        go.Bar(x=future_counts.index, y=future_counts.values,
               name="Next Season Projection", marker_color='gold'),
        row=2, col=2
    )

    fig.update_layout(height=800, title_text="📅 Year Class Analysis & Future Outlook", showlegend=False)
    fig.update_xaxes(title_text="Year", row=1, col=1)
    fig.update_yaxes(title_text="Player Count", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=2)
    fig.update_yaxes(title_text="Position", row=1, col=2)
    fig.update_xaxes(title_text="Year", row=2, col=1)
    fig.update_yaxes(title_text="Average Value", row=2, col=1)
    fig.update_xaxes(title_text="Position", row=2, col=2, tickangle=45)
    fig.update_yaxes(title_text="Player Count", row=2, col=2)

    fig.show()

    # Print year class insights
    print("📅 Year Class Insights:")
    graduating = len(roster_df[roster_df['STATUS'] == 'GRADUATING'])
    total_players = len(roster_df)
    print(f"  • {graduating} players graduating ({graduating/total_players*100:.1f}% of roster)")
    print(f"  • {len(future_roster)} players returning next season")

    # Show positions most affected by graduation
    grad_by_pos = roster_df[roster_df['STATUS'] == 'GRADUATING']['POSITION'].value_counts()
    if len(grad_by_pos) > 0:
        print(f"  • Positions losing most players: {', '.join(grad_by_pos.head(3).index.tolist())}")

📅 Year Class Insights:
  • 21 players graduating (24.7% of roster)
  • 64 players returning next season
  • Positions losing most players: CB, DT, WR


In [ ]:
# 6. Geographic Heatmap - Player Distribution by Location
# This cell demonstrates how to create a US geographic heatmap if you have CITY and STATE columns

def create_geographic_heatmap(df):
    """Create a geographic heatmap showing player distribution across the US."""

    # Check if geographic columns exist
    if 'CITY' not in df.columns or 'STATE' not in df.columns:
        print("⚠️  Geographic heatmap requires 'CITY' and 'STATE' columns in your data")
        print("📝 Sample data structure needed:")
        print("   FIRST NAME | LAST NAME | POSITION | CITY        | STATE")
        print("   John       | Smith     | QB       | Atlanta     | GA")
        print("   Mike       | Johnson   | RB       | Dallas      | TX")
        return None

    try:
        # Count players by state
        state_counts = df['STATE'].value_counts().reset_index()
        state_counts.columns = ['state', 'player_count']

        # Create choropleth map
        fig = go.Figure(data=go.Choropleth(
            locations=state_counts['state'],
            z=state_counts['player_count'],
            locationmode='USA-states',
            colorscale='Reds',
            text=state_counts['state'],
            marker_line_color='white',
            colorbar_title="Players"
        ))

        fig.update_layout(
            title='🗺️ Player Geographic Distribution - US Heatmap',
            geo=dict(
                scope='usa',
                projection=go.layout.geo.Projection(type='albers usa'),
                showlakes=True,
                lakecolor='rgb(255, 255, 255)'
            ),
            height=600
        )

        fig.show()

        # Create city-level scatter plot
        if df['CITY'].notna().sum() > 0:
            # For this demo, we'll use a scatter plot since we'd need lat/lon coordinates for actual city mapping
            city_state_counts = df.groupby(['CITY', 'STATE']).size().reset_index(name='player_count')
            city_state_counts = city_state_counts.sort_values('player_count', ascending=False).head(20)

            fig2 = px.bar(
                city_state_counts.head(15),
                x='player_count',
                y=[f"{row['CITY']}, {row['STATE']}" for _, row in city_state_counts.head(15).iterrows()],
                orientation='h',
                title='🏙️ Top 15 Cities by Player Count',
                labels={'player_count': 'Number of Players', 'y': 'City, State'}
            )

            fig2.update_layout(height=500, yaxis={'categoryorder': 'total ascending'})
            fig2.show()

        # Print summary statistics
        print("🗺️ Geographic Distribution Summary:")
        print(f"  • Total States Represented: {df['STATE'].nunique()}")
        print(f"  • Total Cities Represented: {df['CITY'].nunique()}")
        print(f"  • Top 5 States by Player Count:")

        top_states = df['STATE'].value_counts().head(5)
        for state, count in top_states.items():
            print(f"    - {state}: {count} players")

        return fig

    except Exception as e:
        print(f"❌ Error creating geographic heatmap: {e}")
        return None

def create_recruiting_territory_map(df):
    """Create a map showing recruiting territories and success by region."""

    if 'CITY' not in df.columns or 'STATE' not in df.columns:
        print("⚠️  Recruiting territory map requires 'CITY' and 'STATE' columns")
        return None

    try:
        # Calculate recruiting metrics by state
        state_metrics = df.groupby('STATE').agg({
            'VALUE': ['count', 'mean', 'max'],
            'DEV TRAIT': lambda x: (x == 'ELITE').sum() + (x == 'STAR').sum()
        }).round(2)

        state_metrics.columns = ['player_count', 'avg_value', 'max_value', 'elite_star_count']
        state_metrics = state_metrics.reset_index()

        # Create recruiting success heatmap
        fig = go.Figure(data=go.Choropleth(
            locations=state_metrics['STATE'],
            z=state_metrics['avg_value'],
            locationmode='USA-states',
            colorscale='RdYlGn',
            text=[f"{state}<br>Avg Value: {avg_val}<br>Players: {count}<br>Elite/Star: {elite}"
                  for state, avg_val, count, elite in zip(
                      state_metrics['STATE'],
                      state_metrics['avg_value'],
                      state_metrics['player_count'],
                      state_metrics['elite_star_count']
                  )],
            hovertemplate='%{text}<extra></extra>',
            marker_line_color='white',
            colorbar_title="Avg Player Value"
        ))

        fig.update_layout(
            title='🎯 Recruiting Territory Success Map - Average Player Value by State',
            geo=dict(
                scope='usa',
                projection=go.layout.geo.Projection(type='albers usa'),
                showlakes=True,
                lakecolor='rgb(255, 255, 255)'
            ),
            height=600
        )

        fig.show()

        print("🎯 Recruiting Territory Analysis:")
        print(f"  • Best Recruiting State (Avg Value): {state_metrics.loc[state_metrics['avg_value'].idxmax(), 'STATE']}")
        print(f"  • Most Players from: {state_metrics.loc[state_metrics['player_count'].idxmax(), 'STATE']}")
        print(f"  • Most Elite/Star Players from: {state_metrics.loc[state_metrics['elite_star_count'].idxmax(), 'STATE']}")

        return fig

    except Exception as e:
        print(f"❌ Error creating recruiting territory map: {e}")
        return None

# Example usage with sample data (if geographic columns exist)
if roster_df is not None:
    # Check if geographic data exists
    has_geographic_data = 'CITY' in roster_df.columns and 'STATE' in roster_df.columns

    if has_geographic_data:
        print("✅ Geographic data found! Creating heatmaps...")
        create_geographic_heatmap(roster_df)
        create_recruiting_territory_map(roster_df)
    else:
        print("📊 Geographic Heatmap Demo")
        print("🗺️ To enable geographic visualization, add these columns to your roster CSV:")
        print("   • CITY - Player's hometown city")
        print("   • STATE - Player's home state (use 2-letter abbreviations: TX, CA, FL, etc.)")
        print()
        print("📝 Example data structure:")
        sample_data = pd.DataFrame({
            'FIRST NAME': ['John', 'Mike', 'David', 'Chris', 'Alex'],
            'LAST NAME': ['Smith', 'Johnson', 'Williams', 'Brown', 'Davis'],
            'POSITION': ['QB', 'RB', 'WR', 'LB', 'CB'],
            'CITY': ['Atlanta', 'Dallas', 'Miami', 'Houston', 'Los Angeles'],
            'STATE': ['GA', 'TX', 'FL', 'TX', 'CA'],
            'VALUE': [145.2, 132.1, 128.5, 142.8, 139.4],
            'DEV TRAIT': ['ELITE', 'STAR', 'IMPACT', 'STAR', 'ELITE']
        })
        print(sample_data.to_string(index=False))
        print()
        print("🔧 Once you have this data, the geographic heatmaps will automatically generate!")
        print("📍 Features include:")
        print("   • US State heatmap showing player distribution")
        print("   • Top cities by player count")
        print("   • Recruiting territory success analysis")
        print("   • Average player value by state")

In [ ]:
# Demo: Geographic Heatmap with Sample Data
# This cell creates sample geographic data to demonstrate the heatmap functionality

# Import required libraries if not already imported
try:
    pd
except NameError:
    import pandas as pd
    import numpy as np

print("🗺️ Geographic Heatmap Demo")
print("Creating sample data to demonstrate geographic visualization...")

# Create sample roster data with geographic information
sample_geographic_data = pd.DataFrame({
    'FIRST NAME': [
        'John', 'Mike', 'David', 'Chris', 'Alex', 'Tyler', 'Josh', 'Brandon', 'Austin', 'Jordan',
        'Ryan', 'Kevin', 'Jason', 'Michael', 'Andrew', 'Daniel', 'Kyle', 'Steven', 'Brian', 'James',
        'Matt', 'Tony', 'Jake', 'Nick', 'Eric', 'Tommy', 'Sean', 'Derek', 'Aaron', 'Blake'
    ],
    'LAST NAME': [
        'Smith', 'Johnson', 'Williams', 'Brown', 'Davis', 'Miller', 'Wilson', 'Moore', 'Taylor', 'Anderson',
        'Thomas', 'Jackson', 'White', 'Harris', 'Martin', 'Thompson', 'Garcia', 'Martinez', 'Robinson', 'Clark',
        'Rodriguez', 'Lewis', 'Lee', 'Walker', 'Hall', 'Allen', 'Young', 'Hernandez', 'King', 'Wright'
    ],
    'POSITION': [
        'QB', 'RB', 'WR', 'LB', 'CB', 'TE', 'OL', 'DL', 'QB', 'WR',
        'RB', 'CB', 'LB', 'WR', 'OL', 'DL', 'TE', 'CB', 'LB', 'QB',
        'RB', 'WR', 'OL', 'DL', 'CB', 'LB', 'TE', 'WR', 'RB', 'QB'
    ],
    'CITY': [
        'Atlanta', 'Dallas', 'Miami', 'Houston', 'Los Angeles', 'Phoenix', 'San Antonio', 'Orlando', 'Austin', 'Tampa',
        'Jacksonville', 'Fort Worth', 'Charlotte', 'Nashville', 'Memphis', 'Birmingham', 'Baton Rouge', 'New Orleans', 'Mobile', 'Pensacola',
        'Tallahassee', 'Gainesville', 'Montgomery', 'Huntsville', 'Jackson', 'Shreveport', 'Little Rock', 'Tulsa', 'Oklahoma City', 'Lubbock'
    ],
    'STATE': [
        'GA', 'TX', 'FL', 'TX', 'CA', 'AZ', 'TX', 'FL', 'TX', 'FL',
        'FL', 'TX', 'NC', 'TN', 'TN', 'AL', 'LA', 'LA', 'AL', 'FL',
        'FL', 'FL', 'AL', 'AL', 'MS', 'LA', 'AR', 'OK', 'OK', 'TX'
    ],
    'VALUE': [
        145.2, 132.1, 128.5, 142.8, 139.4, 126.7, 134.1, 147.3, 125.8, 130.2,
        135.6, 122.4, 140.1, 133.7, 127.9, 138.5, 124.3, 131.8, 129.6, 136.4,
        123.1, 141.2, 126.9, 132.5, 137.8, 121.7, 144.6, 128.3, 133.9, 142.1
    ],
    'DEV TRAIT': [
        'ELITE', 'STAR', 'IMPACT', 'STAR', 'ELITE', 'IMPACT', 'STAR', 'ELITE', 'IMPACT', 'STAR',
        'STAR', 'IMPACT', 'ELITE', 'STAR', 'IMPACT', 'STAR', 'IMPACT', 'STAR', 'IMPACT', 'ELITE',
        'IMPACT', 'ELITE', 'IMPACT', 'STAR', 'STAR', 'IMPACT', 'ELITE', 'IMPACT', 'STAR', 'ELITE'
    ],
    'YEAR': [
        'FR', 'SO', 'JR', 'FR', 'SO', 'JR', 'SR', 'FR', 'SO', 'JR',
        'FR', 'SO', 'JR', 'SR', 'FR', 'SO', 'JR', 'FR', 'SO', 'JR',
        'SR', 'FR', 'SO', 'JR', 'FR', 'SO', 'JR', 'SR', 'FR', 'SO'
    ]
})

# Add additional required columns for compatibility
sample_geographic_data['OVERALL'] = (sample_geographic_data['VALUE'] * 0.7).round().astype(int)
sample_geographic_data['BASE OVERALL'] = sample_geographic_data['OVERALL'] - np.random.randint(0, 5, len(sample_geographic_data))

# Create archetype list that matches exactly the DataFrame length
archetypes = ['DUAL THREAT', 'POWER', 'SPEED', 'COVERAGE']
sample_geographic_data['ARCHETYPE'] = [archetypes[i % len(archetypes)] for i in range(len(sample_geographic_data))]

sample_geographic_data = sample_geographic_data.head(30)  # Ensure exactly 30 players

print(f"✅ Created sample roster with {len(sample_geographic_data)} players")
print(f"🗺️ Geographic distribution:")

state_summary = sample_geographic_data['STATE'].value_counts()
for state, count in state_summary.head(5).items():
    print(f"   • {state}: {count} players")

print(f"\n📊 Sample data preview:")
preview_cols = ['FIRST NAME', 'LAST NAME', 'POSITION', 'CITY', 'STATE', 'VALUE']
print(sample_geographic_data[preview_cols].head(8).to_string(index=False))

print(f"\n🎯 Now demonstrating geographic heatmap functionality...")

# Define the geographic heatmap functions for this demo
def create_geographic_heatmap(df):
    """Create a geographic heatmap showing player distribution across the US."""

    # Check if geographic columns exist
    if 'CITY' not in df.columns or 'STATE' not in df.columns:
        print("⚠️  Geographic heatmap requires 'CITY' and 'STATE' columns in your data")
        return None

    try:
        # Count players by state
        state_counts = df['STATE'].value_counts().reset_index()
        state_counts.columns = ['state', 'player_count']

        # Create choropleth map
        fig = go.Figure(data=go.Choropleth(
            locations=state_counts['state'],
            z=state_counts['player_count'],
            locationmode='USA-states',
            colorscale='Reds',
            text=state_counts['state'],
            marker_line_color='white',
            colorbar_title="Players"
        ))

        fig.update_layout(
            title='🗺️ Player Geographic Distribution - US Heatmap',
            geo=dict(
                scope='usa',
                projection=go.layout.geo.Projection(type='albers usa'),
                showlakes=True,
                lakecolor='rgb(255, 255, 255)'
            ),
            height=600
        )        # Add city pins using automatic geocoding with enhanced timeout handling
        if df['CITY'].notna().sum() > 0:
            # Enhanced coordinate lookup function with timeout control and retry logic
            def get_city_coordinates(city, state, cache={}, timeout=10, max_retries=2):
                """Get coordinates for a city using geopy with improved timeout handling."""
                key = f"{city}, {state}"

                if key in cache:
                    return cache[key]

                for attempt in range(max_retries + 1):
                    try:
                        from geopy.geocoders import Nominatim
                        import time

                        # Exponential backoff for retries
                        if attempt > 0:
                            delay = 0.5 * (2 ** attempt)  # 0.5s, 1s, 2s delays
                            time.sleep(delay)
                            print(f"🔄 Retry {attempt} for {city}, {state}")
                        else:
                            time.sleep(0.1)  # Base rate limiting

                        # Configure explicit timeout
                        geolocator = Nominatim(
                            user_agent="cfb_dynasty_analysis",
                            timeout=timeout
                        )
                        location = geolocator.geocode(f"{city}, {state}, USA")

                        if location:
                            coords = (location.latitude, location.longitude)
                            cache[key] = coords
                            if attempt > 0:
                                print(f"✅ Successfully geocoded {city}, {state} on retry {attempt}")
                            return coords

                    except Exception as e:
                        error_msg = str(e).lower()
                        if ("timed out" in error_msg or "timeout" in error_msg) and attempt < max_retries:
                            print(f"⏱️ Timeout for {city}, {state} - retrying in {0.5 * (2 ** (attempt + 1))}s...")
                            continue
                        elif attempt == max_retries:
                            print(f"⚠️  Failed to geocode {city}, {state} after {max_retries + 1} attempts: {e}")
                            break
                        else:
                            print(f"⚠️  Could not geocode {city}, {state}: {e}")
                            break

                cache[key] = None
                return None

            city_state_counts = df.groupby(['CITY', 'STATE']).size().reset_index(name='player_count')

            # Get coordinates for all cities automatically
            city_lats, city_lons, city_names, city_counts, city_texts = [], [], [], [], []
            print(f"🗺️ Geocoding {len(city_state_counts)} cities...")

            for _, row in city_state_counts.iterrows():
                city_name = row['CITY']
                state_name = row['STATE']
                coords = get_city_coordinates(city_name, state_name)

                if coords:
                    lat, lon = coords
                    city_lats.append(lat)
                    city_lons.append(lon)
                    city_names.append(f"{city_name}, {state_name}")
                    city_counts.append(row['player_count'])
                    city_texts.append(f"{city_name}, {state_name}<br>{row['player_count']} player(s)")

            # Add city pins to the map
            if city_lats:
                fig.add_trace(go.Scattergeo(
                    locationmode='USA-states',
                    lon=city_lons,
                    lat=city_lats,
                    text=city_texts,
                    mode='markers',
                    marker=dict(
                        size=[count * 3 + 8 for count in city_counts],  # Scale marker size by player count
                        color='darkblue',
                        opacity=0.8,
                        line=dict(width=2, color='white'),
                        symbol='circle'
                    ),
                    name='Cities',
                    hovertemplate='%{text}<extra></extra>'
                ))
                print(f"📍 Successfully plotted {len(city_lats)} cities on map")

        fig.show()

        # Create city-level bar chart
        if df['CITY'].notna().sum() > 0:
            city_state_counts = df.groupby(['CITY', 'STATE']).size().reset_index(name='player_count')
            city_state_counts = city_state_counts.sort_values('player_count', ascending=False).head(20)

            fig2 = px.bar(
                city_state_counts.head(15),
                x='player_count',
                y=[f"{row['CITY']}, {row['STATE']}" for _, row in city_state_counts.head(15).iterrows()],
                orientation='h',
                title='🏙️ Top 15 Cities by Player Count',
                labels={'player_count': 'Number of Players', 'y': 'City, State'}
            )

            fig2.update_layout(height=500, yaxis={'categoryorder': 'total ascending'})
            fig2.show()

        # Print summary statistics
        print("🗺️ Geographic Distribution Summary:")
        print(f"  • Total States Represented: {df['STATE'].nunique()}")
        print(f"  • Total Cities Represented: {df['CITY'].nunique()}")
        print(f"  • Top 5 States by Player Count:")

        top_states = df['STATE'].value_counts().head(5)
        for state, count in top_states.items():
            print(f"    - {state}: {count} players")

        return fig

    except Exception as e:
        print(f"❌ Error creating geographic heatmap: {e}")
        return None

def create_recruiting_territory_map(df):
    """Create a map showing recruiting territories and success by region."""

    if 'CITY' not in df.columns or 'STATE' not in df.columns:
        print("⚠️  Recruiting territory map requires 'CITY' and 'STATE' columns")
        return None

    try:
        # Calculate recruiting metrics by state
        state_metrics = df.groupby('STATE').agg({
            'VALUE': ['count', 'mean', 'max'],
            'DEV TRAIT': lambda x: (x == 'ELITE').sum() + (x == 'STAR').sum()
        }).round(2)

        state_metrics.columns = ['player_count', 'avg_value', 'max_value', 'elite_star_count']
        state_metrics = state_metrics.reset_index()

        # Create recruiting success heatmap
        fig = go.Figure(data=go.Choropleth(
            locations=state_metrics['STATE'],
            z=state_metrics['avg_value'],
            locationmode='USA-states',
            colorscale='RdYlGn',
            text=[f"{state}<br>Avg Value: {avg_val}<br>Players: {count}<br>Elite/Star: {elite}"
                  for state, avg_val, count, elite in zip(
                      state_metrics['STATE'],
                      state_metrics['avg_value'],
                      state_metrics['player_count'],
                      state_metrics['elite_star_count']
                  )],
            hovertemplate='%{text}<extra></extra>',
            marker_line_color='white',
            colorbar_title="Avg Player Value"
        ))

        fig.update_layout(
            title='🎯 Recruiting Territory Success Map - Average Player Value by State',
            geo=dict(
                scope='usa',
                projection=go.layout.geo.Projection(type='albers usa'),
                showlakes=True,
                lakecolor='rgb(255, 255, 255)'
            ),
            height=600
        )

        fig.show()

        print("🎯 Recruiting Territory Analysis:")
        print(f"  • Best Recruiting State (Avg Value): {state_metrics.loc[state_metrics['avg_value'].idxmax(), 'STATE']}")
        print(f"  • Most Players from: {state_metrics.loc[state_metrics['player_count'].idxmax(), 'STATE']}")
        print(f"  • Most Elite/Star Players from: {state_metrics.loc[state_metrics['elite_star_count'].idxmax(), 'STATE']}")

        return fig

    except Exception as e:
        print(f"❌ Error creating recruiting territory map: {e}")
        return None

# Import required plotly modules if not already available
try:
    # Test if plotly modules are already imported
    go
    px
except NameError:
    # Import plotly modules
    try:
        import plotly.graph_objects as go
        import plotly.express as px
        print("📦 Plotly modules imported successfully!")
    except ImportError:
        print("❌ Plotly not installed. Installing now...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'plotly'])
        import plotly.graph_objects as go
        import plotly.express as px
        print("✅ Plotly installed and imported successfully!")

# Demonstrate the geographic heatmap functions with sample data
create_geographic_heatmap(sample_geographic_data)
create_recruiting_territory_map(sample_geographic_data)

🗺️ Geographic Heatmap Demo
Creating sample data to demonstrate geographic visualization...
✅ Created sample roster with 30 players
🗺️ Geographic distribution:
   • FL: 7 players
   • TX: 6 players
   • AL: 4 players
   • LA: 3 players
   • TN: 2 players

📊 Sample data preview:
FIRST NAME LAST NAME POSITION        CITY STATE  VALUE
      John     Smith       QB     Atlanta    GA  145.2
      Mike   Johnson       RB      Dallas    TX  132.1
     David  Williams       WR       Miami    FL  128.5
     Chris     Brown       LB     Houston    TX  142.8
      Alex     Davis       CB Los Angeles    CA  139.4
     Tyler    Miller       TE     Phoenix    AZ  126.7
      Josh    Wilson       OL San Antonio    TX  134.1
   Brandon     Moore       DL     Orlando    FL  147.3

🎯 Now demonstrating geographic heatmap functionality...
🗺️ Geocoding 30 cities...
📍 Successfully plotted 30 cities on map
📍 Successfully plotted 30 cities on map


🗺️ Geographic Distribution Summary:
  • Total States Represented: 12
  • Total Cities Represented: 30
  • Top 5 States by Player Count:
    - FL: 7 players
    - TX: 6 players
    - AL: 4 players
    - LA: 3 players
    - TN: 2 players


🎯 Recruiting Territory Analysis:
  • Best Recruiting State (Avg Value): GA
  • Most Players from: FL
  • Most Elite/Star Players from: FL


In [ ]:
# 5. Top Performers and Cut Candidates Analysis
if roster_df is not None:
    # Create top performers vs cut candidates comparison
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Top 15 Players by Value', 'Development Trait vs Rating',
                       'Cut Candidates', 'At-Risk Players'),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "table"}, {"type": "table"}]]
    )

    # Top performers
    top_players = roster_df.nlargest(15, 'VALUE')
    player_names = top_players['FIRST NAME'] + ' ' + top_players['LAST NAME']

    fig.add_trace(
        go.Bar(x=player_names, y=top_players['VALUE'],
               name="Top Players",
               marker_color=top_players['VALUE'],
               marker_colorscale='Viridis',
               text=top_players['POSITION'],
               textposition='auto'),
        row=1, col=1
    )

    # Development trait vs rating scatter
    dev_trait_colors = {'ELITE': '#2E8B57', 'STAR': '#32CD32', 'IMPACT': '#FFD700', 'NORMAL': '#808080'}
    for trait in roster_df['DEV TRAIT'].unique():
        trait_data = roster_df[roster_df['DEV TRAIT'] == trait]
        fig.add_trace(
            go.Scatter(x=trait_data['BASE OVERALL'], y=trait_data['VALUE'],
                      mode='markers', name=trait,
                      marker=dict(color=dev_trait_colors.get(trait, '#808080'), size=8)),
            row=1, col=2
        )

    # Cut candidates table
    cut_candidates = roster_df[roster_df['STATUS'] == 'CUT'][
        ['FIRST NAME', 'LAST NAME', 'POSITION', 'YEAR', 'RATING', 'VALUE']
    ].round(1)

    if len(cut_candidates) > 0:
        fig.add_trace(
            go.Table(
                header=dict(values=list(cut_candidates.columns), fill_color='lightcoral'),
                cells=dict(values=[cut_candidates[col] for col in cut_candidates.columns],
                          fill_color='white')
            ),
            row=2, col=1
        )

    # At-risk players table
    at_risk = roster_df[roster_df['STATUS'] == 'AT RISK'][
        ['FIRST NAME', 'LAST NAME', 'POSITION', 'YEAR', 'RATING', 'VALUE']
    ].round(1)

    if len(at_risk) > 0:
        fig.add_trace(
            go.Table(
                header=dict(values=list(at_risk.columns), fill_color='gold'),
                cells=dict(values=[at_risk[col] for col in at_risk.columns],
                          fill_color='white')
            ),
            row=2, col=2
        )

    fig.update_layout(height=800, title_text="⭐ Player Performance Analysis", showlegend=True)
    fig.update_xaxes(title_text="Player", row=1, col=1, tickangle=45)
    fig.update_yaxes(title_text="Player Value", row=1, col=1)
    fig.update_xaxes(title_text="Base Overall", row=1, col=2)
    fig.update_yaxes(title_text="Player Value", row=1, col=2)

    fig.show()

    # Print key insights
    print("⭐ Performance Insights:")
    print(f"  • Top Player: {roster_df.loc[roster_df['VALUE'].idxmax(), 'FIRST NAME']} {roster_df.loc[roster_df['VALUE'].idxmax(), 'LAST NAME']} ({roster_df['VALUE'].max():.1f} value)")
    print(f"  • Elite Dev Traits: {len(roster_df[roster_df['DEV TRAIT'] == 'ELITE'])} players")
    print(f"  • Star Dev Traits: {len(roster_df[roster_df['DEV TRAIT'] == 'STAR'])} players")
    print(f"  • Players to Cut: {len(cut_candidates)}")
    print(f"  • Players At Risk: {len(at_risk)}")

KeyError: "['RATING'] not in index"

## 💾 Export Results

Save your analysis results to CSV files for future reference or sharing.

In [ ]:
# Export Analysis Results
if roster_df is not None and recruiting_plan is not None:
    # Create output directory
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    # Export processed roster with player values
    roster_export = roster_df.drop('Best at Position', axis=1, errors='ignore')
    player_values_path = os.path.join(data_folder, 'player_values_analysis.csv')
    roster_export.to_csv(player_values_path, index=False)

    # Export recruiting plan
    recruiting_plan_path = os.path.join(data_folder, 'recruiting_plan_analysis.csv')
    recruiting_plan.to_csv(recruiting_plan_path, index=False)

    # Export detailed position analysis
    position_analysis = []
    for pos in default_position_requirements.keys():
        pos_data = roster_df[roster_df['POSITION'] == pos]
        if len(pos_data) > 0:
            analysis = {
                'Position': pos,
                'Total_Players': len(pos_data),
                'Avg_Rating': pos_data['RATING'].mean(),
                'Avg_Value': pos_data['VALUE'].mean(),
                'Top_Player_Value': pos_data['VALUE'].max(),
                'Top_Player_Name': f"{pos_data.loc[pos_data['VALUE'].idxmax(), 'FIRST NAME']} {pos_data.loc[pos_data['VALUE'].idxmax(), 'LAST NAME']}",
                'Elite_Dev_Count': len(pos_data[pos_data['DEV TRAIT'] == 'ELITE']),
                'Star_Dev_Count': len(pos_data[pos_data['DEV TRAIT'] == 'STAR']),
                'Graduating_Count': len(pos_data[pos_data['STATUS'] == 'GRADUATING']),
                'Cut_Candidates': len(pos_data[pos_data['STATUS'] == 'CUT']),
                'At_Risk_Count': len(pos_data[pos_data['STATUS'] == 'AT RISK'])
            }
            position_analysis.append(analysis)

    position_analysis_df = pd.DataFrame(position_analysis)
    position_analysis_path = os.path.join(data_folder, 'position_analysis_detailed.csv')
    position_analysis_df.to_csv(position_analysis_path, index=False)

    print("💾 Export completed successfully!")
    print(f"📁 Files saved to: {data_folder}")
    print(f"  📊 Player Values: {os.path.basename(player_values_path)}")
    print(f"  🎯 Recruiting Plan: {os.path.basename(recruiting_plan_path)}")
    print(f"  📋 Position Analysis: {os.path.basename(position_analysis_path)}")

    # Show summary of what was exported
    print(f"\n📈 Export Summary:")
    print(f"  • {len(roster_export)} player records with values and status")
    print(f"  • {len(recruiting_plan)} position recruiting priorities")
    print(f"  • {len(position_analysis_df)} detailed position breakdowns")

else:
    print("❌ Cannot export - no data available. Please run the analysis cells above first.")

## 🔍 Interactive Data Exploration

Use these cells to explore your data interactively. Modify the code below to filter and analyze specific aspects of your roster.

In [ ]:
# Interactive Exploration Examples

# Example 1: Filter by specific position (change 'QB' to any position)
if roster_df is not None:
    position_to_analyze = 'QB'  # Change this to any position
    position_players = roster_df[roster_df['POSITION'] == position_to_analyze].copy()

    if len(position_players) > 0:
        print(f"📊 {position_to_analyze} Analysis:")
        print(f"  • Total Players: {len(position_players)}")
        print(f"  • Average Rating: {position_players['RATING'].mean():.1f}")
        print(f"  • Average Value: {position_players['VALUE'].mean():.1f}")
        print(f"  • Top Player: {position_players.loc[position_players['VALUE'].idxmax(), 'FIRST NAME']} {position_players.loc[position_players['VALUE'].idxmax(), 'LAST NAME']} ({position_players['VALUE'].max():.1f})")

        # Show detailed breakdown
        print(f"\n📋 Detailed {position_to_analyze} Roster:")
        display_cols = ['RS', 'FIRST NAME', 'LAST NAME', 'YEAR', 'OVERALL', 'BASE OVERALL', 'ARCHETYPE', 'DEV TRAIT', 'VALUE', 'STATUS']
        print(position_players[display_cols].sort_values('VALUE', ascending=False).to_string(index=False))
    else:
        print(f"❌ No players found at {position_to_analyze} position")
else:
    print("❌ No roster data available. Please run the data loading cells first.")

In [ ]:
# Example 2: Custom Filtering - High Value Underclassmen
if roster_df is not None:
    # Find high-value underclassmen (FR/SO with value > 120)
    underclassmen = roster_df[
        (roster_df['YEAR'].isin(['FR', 'SO', 'FR (RS)', 'SO (RS)'])) &
        (roster_df['VALUE'] > 120)
    ].copy()

    if len(underclassmen) > 0:
        print("⭐ High-Value Underclassmen (Future Stars):")
        print(f"Found {len(underclassmen)} underclassmen with value > 120")

        # Create visualization
        fig = px.scatter(underclassmen, x='BASE OVERALL', y='VALUE',
                        color='DEV TRAIT', size='OVERALL',
                        hover_data=['FIRST NAME', 'LAST NAME', 'POSITION', 'YEAR'],
                        title='🌟 Future Stars: High-Value Underclassmen')
        fig.show()

        # Show top prospects
        top_prospects = underclassmen.nlargest(10, 'VALUE')
        display_cols = ['FIRST NAME', 'LAST NAME', 'POSITION', 'YEAR', 'RATING', 'DEV TRAIT', 'VALUE']
        print(f"\n🏆 Top 10 Future Prospects:")
        print(top_prospects[display_cols].to_string(index=False))
    else:
        print("🔍 No high-value underclassmen found (try lowering the value threshold)")

# Example 3: Position Group Analysis
if roster_df is not None:
    print(f"\n📊 Position Group Summary:")

    # Group positions by unit
    position_groups = {
        'Offense': ['QB', 'HB', 'FB', 'WR', 'TE', 'LT', 'LG', 'C', 'RG', 'RT'],
        'Defense': ['LE', 'RE', 'DT', 'LOLB', 'MLB', 'ROLB', 'CB', 'FS', 'SS'],
        'Special Teams': ['K', 'P']
    }

    for group_name, positions in position_groups.items():
        group_players = roster_df[roster_df['POSITION'].isin(positions)]
        if len(group_players) > 0:
            avg_value = group_players['VALUE'].mean()
            print(f"  {group_name}: {len(group_players)} players, Avg Value: {avg_value:.1f}")
        else:
            print(f"  {group_name}: No players found")

## 📋 Summary & Next Steps

This notebook has provided a comprehensive analysis of your CFB Dynasty roster including:

- ✅ **Player Valuations** - Each player's calculated value based on ratings, development potential, and remaining years
- ✅ **Roster Composition** - Visual breakdown of your team's strengths and weaknesses by position
- ✅ **Recruiting Strategy** - Data-driven priorities for recruiting based on position needs and current grades
- ✅ **Future Outlook** - Analysis of year classes and projected roster changes
- ✅ **Performance Insights** - Identification of top performers and players at risk

### 🎯 Key Actions to Take:

1. **Review High Priority Positions** - Focus recruiting efforts on positions marked as HIGH priority
2. **Evaluate Cut Candidates** - Consider cutting players with status "CUT" to make roster space
3. **Monitor At-Risk Players** - Keep an eye on players with status "AT RISK" for potential development
4. **Plan for Graduation** - Prepare for the loss of graduating seniors by recruiting replacements

### 📁 Exported Files:
- `player_values_analysis.csv` - Complete roster with calculated values and status
- `recruiting_plan_analysis.csv` - Position-by-position recruiting priorities  
- `position_analysis_detailed.csv` - Detailed breakdown of each position group

---

*This analysis was generated using the CFB Dynasty Roster Analysis notebook. Modify the parameters and filters above to customize the analysis for your specific needs.*